## Notebook description/goals


In [9]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [10]:
import sys
sys.path.append('/data/tjarniko/MEOPAR/mocsy')
sys.path.append('/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/VIZ')
import mocsy
import VIZ_plots as VP

import time
import numpy as np
import matplotlib.pyplot as plt
from oct2py import octave
import seawater as sw
from __future__ import print_function
from numpy import *
from scipy import *
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import reload
plt.style.use('seaborn-white')
import netCDF4 as nc
import numpy as np
import cmocean as cm

""
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

%matplotlib inline

In [11]:
reload(VP)

<module 'VIZ_plots' from '/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/VIZ/VIZ_plots.py'>

## Printouts of diagnostics from point 250, 250 at timestep 0 

In [12]:

td = nc.Dataset('/data/tjarniko/results/a31_6/SalishSea_1ts_20171026_20171026_ptrc_T.nc')
print(td)
tdg = nc.Dataset('/data/tjarniko/results/a30_1/SalishSea_1ts_20171026_20171026_grid_T.nc')
#print(tdg)

temp = tdg.variables['votemper'][:]
sal = tdg.variables['vosaline'][:]
print('temp: ', temp[0,0,250,250])
print('sal: ', sal[0,0,250,250])

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    name: SalishSea_1ts_20171026_20171026
    description: biogeochemical variables
    title: biogeochemical variables
    Conventions: CF-1.6
    timeStamp: 2018-Aug-30 19:10:53 GMT
    uuid: ac565765-d369-4be7-9acd-b2c3d444bfc6
    dimensions(sizes): axis_nbounds(2), x(398), y(898), nvertex(4), deptht(40), time_counter(3)
    variables(dimensions): float32 nav_lat(y,x), float32 nav_lon(y,x), float32 bounds_lon(y,x,nvertex), float32 bounds_lat(y,x,nvertex), float32 area(y,x), float32 deptht(deptht), float32 deptht_bounds(deptht,axis_nbounds), float32 nitrate(time_counter,deptht,y,x), float64 time_centered(time_counter), float64 time_centered_bounds(time_counter,axis_nbounds), float64 time_counter(time_counter), float64 time_counter_bounds(time_counter,axis_nbounds), float32 ammonium(time_counter,deptht,y,x), float32 silicon(time_counter,deptht,y,x), float32 diatoms(time_counter,deptht,y,x), float32 

In [13]:
f_ph = td.variables['f_ph'][:]
f_pco2w = td.variables['f_pco2w'][:]
# f_fco2w = td.variables['f_fco2w'][:]
# f_co2w = td.variables['f_co2w'][:]
# f_hco3 = td.variables['f_hco3'][:]

#f_co3 = td.variables['f_co3'][:]
f_omarg = td.variables['f_omarg'][:]
# f_omcal = td.variables['f_omcal'][:]
# f_BetaD = td.variables['f_BetaD'][:]
# f_rhosw = td.variables['f_rhosw'][:]

# f_opres = td.variables['f_opres'][:]
# f_insitut = td.variables['f_insitut'][:]
# f_pco2atm = td.variables['f_pco2atm'][:]
# f_fco2atm = td.variables['f_fco2atm'][:]
# f_schmidtco2 = td.variables['f_schmidtco2'][:]

# f_kwco2 = td.variables['f_kwco2'][:]
# f_K0 = td.variables['f_K0'][:]
# f_co2starair = td.variables['f_co2starair'][:]
f_co2flux = td.variables['f_co2flux'][:]
# f_dpco2 = td.variables['f_dpco2'][:]


dar = [f_ph, f_pco2w, f_omarg, f_co2flux]

darn = ['f_ph', 'f_pco2w', 'f_omarg', 'f_co2flux']

for i in range(0,len(dar)):
    print(darn[i])
    print(dar[i][0,250,250])

f_ph
7.74114
f_pco2w
803.021
f_omarg
0.996381
f_co2flux
-2.96733e-05


## schmidt
    subroutine schmidt_co2(pt, N, co2_schmidt)

      USE mocsy_singledouble

      implicit none
    !
      INTEGER, INTENT(in) :: N
      real(kind=wp), INTENT(in),  DIMENSION(N) :: pt
      real(kind=wp), INTENT(out), DIMENSION(N) :: co2_schmidt
    !
      real(kind=wp)              :: a0, a1, a2, a3, a4

      data a0 /    2116.8 /
      data a1 /   -136.25 /
      data a2 /    4.7353 /
      data a3 / -0.092307 /
      data a4 / 0.0007555 /
    !
    ! Wanninkhof (1992)
    !     co2_schmidt = a0 + pt*(a1 + pt*(a2 + pt*a3))

    ! Wanninkhof (2014) adds in an extra term
      co2_schmidt = a0 + pt*(a1 + pt*(a2 + pt*(a3 + pt*a4)))

      return
      end subroutine schmidt_co2

    SUBROUTINE x2pCO2atm(xCO2, temp, salt, Patm, N, pCO2atm)
      !    Purpose:
      !    Compute pCO2atm from arrays of xCO2, in situ T, S, & atm pressure

      USE mocsy_singledouble

      IMPLICIT NONE

      !> number of records
      INTEGER, intent(in) :: N

    ! INPUT variables
      !> mole fraction of CO2 [ppm]
      REAL(kind=wp), INTENT(in), DIMENSION(N) :: xCO2
      !> in situ temperature [C]
      REAL(kind=wp), INTENT(in), DIMENSION(N) :: temp
      !> salinity [psu]
      REAL(kind=wp), INTENT(in), DIMENSION(N) :: salt
      !> atmospheric pressure [atm]
      REAL(kind=wp), INTENT(in), DIMENSION(N) :: Patm
    !f2py optional , depend(temp) :: n=len(temp)

    ! OUTPUT variables:
      !> oceanic partial pressure of CO2 [uatm]
      REAL(kind=wp), INTENT(out), DIMENSION(N) :: pCO2atm

    ! LOCAL variables:
      REAL(kind=wp) :: dxCO2, dPatm
      REAL(kind=wp), DIMENSION(N) :: pH20
      REAL(kind=wp) :: dpCO2atm

      INTEGER :: i

    ! Compute vapor pressure of seawater [in atm]
      call vapress(temp, salt, N, pH20)

      DO i = 1,N
         dxCO2     = DBLE(xCO2(i))
         dPatm     = DBLE(Patm(i))
         dpCO2atm = (dPatm - pH20(i)) * dxCO2
         pCO2atm(i) = REAL(dpCO2atm)
      END DO

      RETURN
    END SUBROUTINE x2pCO2atm

## Schmidt number - python rewrite

In [14]:
pt = temp[0,0,250,250]

a0 = 2116.8 
a1 =  -136.25 
a2 =   4.7353 
a3 = -0.092307 
a4 = 0.0007555 

# Wanninkhof (1992)
#     co2_schmidt = a0 + pt*(a1 + pt*(a2 + pt*a3))

# Wanninkhof (2014) adds in an extra term
co2_schmidt = a0 + pt*(a1 + pt*(a2 + pt*(a3 + pt*a4)))
print(co2_schmidt)

974.654599734


##  Compute pCO2atm from xCO2 - fortran

    SUBROUTINE x2pCO2atm(xCO2, temp, salt, Patm, N, pCO2atm)
      !    Purpose:
      !    Compute pCO2atm from arrays of xCO2, in situ T, S, & atm pressure

      USE mocsy_singledouble

      IMPLICIT NONE

      !> number of records
      INTEGER, intent(in) :: N

    ! INPUT variables
      !> mole fraction of CO2 [ppm]
      REAL(kind=wp), INTENT(in), DIMENSION(N) :: xCO2
      !> in situ temperature [C]
      REAL(kind=wp), INTENT(in), DIMENSION(N) :: temp
      !> salinity [psu]
      REAL(kind=wp), INTENT(in), DIMENSION(N) :: salt
      !> atmospheric pressure [atm]
      REAL(kind=wp), INTENT(in), DIMENSION(N) :: Patm
    !f2py optional , depend(temp) :: n=len(temp)

    ! OUTPUT variables:
      !> oceanic partial pressure of CO2 [uatm]
      REAL(kind=wp), INTENT(out), DIMENSION(N) :: pCO2atm

    ! LOCAL variables:
      REAL(kind=wp) :: dxCO2, dPatm
      REAL(kind=wp), DIMENSION(N) :: pH20
      REAL(kind=wp) :: dpCO2atm

      INTEGER :: i

    ! Compute vapor pressure of seawater [in atm]
      call vapress(temp, salt, N, pH20)

      DO i = 1,N
         dxCO2     = DBLE(xCO2(i))
         dPatm     = DBLE(Patm(i))
         dpCO2atm = (dPatm - pH20(i)) * dxCO2
         pCO2atm(i) = REAL(dpCO2atm)
      END DO

      RETURN
    END SUBROUTINE x2pCO2atm

    ! --------------------------------------------------------------------
    !  Calculate seawater vapor pressure
    ! --------------------------------------------------------------------
    !
    !>    Compute vapor pressure of seawater (atm) following preocedure from Weiss & Price (1980)
    SUBROUTINE vapress(temp, salt, N, vpsw)
      !    Purpose:
      !    Compute vapor pressure of seawater (atm) following preocedure from Weiss & Price (1980)

      USE mocsy_singledouble
      IMPLICIT NONE

      !> number of records
      INTEGER, intent(in) :: N

    ! INPUT variables
      !> in situ temperature [C]
      REAL(kind=wp), INTENT(in), DIMENSION(N) :: temp
      !> salinity [psu]
      REAL(kind=wp), INTENT(in), DIMENSION(N) :: salt
    !f2py optional , depend(temp) :: n=len(temp)

    ! OUTPUT variables:
      !> vapor pressure of seawater [atm]
      REAL(kind=wp), INTENT(out), DIMENSION(N) :: vpsw

    ! LOCAL variables:
      REAL(kind=wp) :: tk, dsalt

      INTEGER :: i

      DO i = 1,N
         dsalt = DBLE(salt(i))
         tk = 273.15d0 + DBLE(temp(i))     !Absolute temperature (Kelvin)
         vpsw(i) = exp(24.4543d0 - 67.4509d0*(100.0d0/tk) - 4.8489d0*log(tk/100) - 0.000544d0*dsalt)
      END DO

      RETURN
    END SUBROUTINE vapress

## Python rewrite of SUBROUTINE x2pCO2atm

In [15]:


temp = 12.8125
salt = 30.6875
patm = 1
xco2 = 412

def x2pco2atm(xco2,temp,salt,patm):    
    
    pH20 = vapress(temp,salt)
    
    pco2atm = (patm - pH20)  * xco2
    
    return(pco2atm)


def vapress(temp,salt):


    tk = 273.15 + temp    #!Absolute temperature (Kelvin)
    vpsw = exp(24.4543 - 67.4509*(100.0/tk) - 4.8489*np.log(tk/100) - 0.000544*salt)

    return vpsw



In [16]:
x2pco2atm(xco2,temp,salt,patm)

406.0901958978028